In [ ]:
import numpy as np
import psi4
import fortecubeview
from prop_sapt.cubes import make_cube
from prop_sapt.utils import prepare_path

In [ ]:
!pwd # we should be in `examples`

In [ ]:
# specify geometry in Psi4 format
GEO = """
symmetry c1
no_com
no_reorient
units bohr
0 1
H    0.000000   0.000000  -0.724500
F    0.000000   0.000000   0.724500
"""

# specify memory and threads
MEMORY = "2 GB"
THREADS = 2

# specify basis sets
BASIS = "aug-cc-pvtz"
DF_BASIS = "aug-cc-pvtz"

# specify options
OPTIONS = {
    # "option": "value",
    "basis": BASIS,
    "DF_BASIS_SCF": DF_BASIS + "-jkfit",
    "scf_type": "direct",
}

# specify output and resultS filenames
OUTPUT_FILE_PATH = "output.dat"
RESULTS_FILE_PATH = "results.csv"

In [ ]:
# NOTE: this cell make take around 30 seconds to run

### Psi4 options
psi4.set_memory(MEMORY)
psi4.set_num_threads(THREADS)
psi4.core.set_output_file(OUTPUT_FILE_PATH, False)
psi4.set_options(OPTIONS)

np.set_printoptions(precision=3, suppress=True)

### Perform SCF
molecule = psi4.geometry(GEO)
energy, wfn = psi4.energy("scf", molecule=molecule, return_wfn=True)

### Grab occupied orbitals
ndocc = wfn.doccpi()[0]
Co = wfn.Ca().to_array()[:, :ndocc]

prepare_path("hf-orbital-cubes/")

for i in range(ndocc):

    cube = make_cube(molecule, Co[:, [i]], obj_type="orbital", grid_overage=6.0)
    cube.save(filename=f"hf-orbital-cubes/orbital_{i+1}.cube")

### End calculations
psi4.core.clean()

In [ ]:
fortecubeview.plot("hf-orbital-cubes", opacity=0.75, sumlevel=0.98)